In [1]:
import tensorflow as tf

这三种方式所定义的变量具有相同的类型。而且只有 tf.get_variable() 创建的变量之间会发生命名冲突。在实际使用中，三种创建变量方式的用途也是分工非常明确的。其中

    tf.placeholder() 占位符。* trainable==False *
    tf.Variable() 一般变量用这种方式定义。 * 可以选择 trainable 类型 *
    tf.get_variable() 一般都是和 tf.variable_scope() 配合使用，从而实现变量共享的功能。 * 可以选择 trainable 类型 *


In [2]:
# 1.placeholder
v1 = tf.placeholder(tf.float32, shape=[2,3,4]) 
print(v1.name)
v1 = tf.placeholder(tf.float32, shape=[2,3,4], name='ph') 
print(v1.name) 
v11 = tf.placeholder(tf.float32, shape=[2,3,4], name='ph') 
print(v11.name) 
print(type(v1)) 
print(v1)

Placeholder:0
ph:0
ph_1:0
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("ph:0", shape=(2, 3, 4), dtype=float32)


In [3]:
v2 = tf.Variable([1,2], dtype=tf.float32) 
print(v2.name)
v2 = tf.Variable([1,2], dtype=tf.float32, name='V') 
print(v2.name)
v22 = tf.Variable([1,2], dtype=tf.float32, name='V') 
print(v22.name)
print(type(v2))
print(v2)


Variable:0
V:0
V_1:0
<class 'tensorflow.python.ops.variables.Variable'>
<tf.Variable 'V:0' shape=(2,) dtype=float32_ref>


In [8]:
scope.reuse_variables()
v3 = tf.get_variable(name='gv', shape=[])  
print(v3.name)
v4 = tf.get_variable(name='1gv', shape=[2])
print(v4.name)

NameError: name 'scope' is not defined

In [5]:
vs = tf.trainable_variables()
print(len(vs))
for v in vs:
    print(v)

5
<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'V:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'V_1:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'gv:0' shape=() dtype=float32_ref>
<tf.Variable '1gv:0' shape=(2,) dtype=float32_ref>


tf.name_scope() 并不会对 tf.get_variable() 创建的变量有任何影响。 tf.name_scope() 主要是用来管理命名空间的，这样子让我们的整个模型更加有条理。而 tf.variable_scope() 的作用是为了实现变量共享，它和 tf.get_variable() 来完成变量共享的功能。


In [6]:
with tf.name_scope('nsc1'):
    v1 = tf.Variable([1], name='v1') 
    with tf.variable_scope('vsc1'): 
        v2 = tf.Variable([1], name='v2') 
        v3 = tf.get_variable(name='v3', shape=[]) 
        print( 'v1.name: ', v1.name )
        print( 'v2.name: ', v2.name )
        print( 'v3.name: ', v3.name)


v1.name:  nsc1/v1:0
v2.name:  nsc1/vsc1/v2:0
v3.name:  vsc1/v3:0


In [9]:
with tf.variable_scope("a_variable_scope") as scope:
    initializer = tf.constant_initializer(value=3)
    var3 = tf.get_variable(name='var3', shape=[1], dtype=tf.float32, initializer=initializer)
    scope.reuse_variables()
    var3_reuse = tf.get_variable(name='var3',)
    var4 = tf.Variable(name='var4', initial_value=[4], dtype=tf.float32)
    var4_reuse = tf.Variable(name='var4', initial_value=[4], dtype=tf.float32)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var3.name)            # a_variable_scope/var3:0
    print(sess.run(var3))       # [ 3.]
    print(var3_reuse.name)      # a_variable_scope/var3:0
    print(sess.run(var3_reuse)) # [ 3.]
    print(var4.name)            # a_variable_scope/var4:0
    print(sess.run(var4))       # [ 4.]
    print(var4_reuse.name)      # a_variable_scope/var4_1:0
    print(sess.run(var4_reuse)) # [ 4.]
    


a_variable_scope/var3:0
[3.]
a_variable_scope/var3:0
[3.]
a_variable_scope/var4:0
[4.]
a_variable_scope/var4_1:0
[4.]
